# Experiments名を指定して、学習ジョブを発行

Python 3(PyTorch 1.10 Python 3.8 CPU Optimized)で動作確認

# MNIST Training using PyTorch

参考：wandb

https://github.com/wandb/examples/blob/master/examples/pytorch/pytorch-mnist-sagemaker/pytorch_mnist.ipynb

SageMaker sample

https://github.com/aws/amazon-sagemaker-examples/blob/main/hyperparameter_tuning/pytorch_mnist/hpo_pytorch_mnist.ipynb

In [ ]:
!pip install ipywidgets widgetsnbextension # studioの場合は必要。要カーネル再起動

* torch / torchvision : 1.11.0 / 0.5.0 -> NG
* torch / torchvision : 1.10.0 / 0.5.0 -> NG
* torch / torchvision : 1.5.0 / 0.5.0 -> OK
* torch / torchvision : 1.7.0 / 0.5.0 -> OK
* torch / torchvision : 1.9.0 / 0.10.0 -> NG
* torch / torchvision : 1.10.2 / 0.11.3 -> NG
* torch / torchvision : 1.9.1 / 0.10.1 -> NG
* torch / torchvision : 1.8.0 / 0.9.0 -> OK

In [ ]:
!pip install -U torch==1.8.0 # 1.4.0で動作確認済み # 1.11.0 NG

In [ ]:
!pip install torchvision==0.9.0 # 0.5.0で動作確認済み

In [ ]:
!pip freeze | grep torch

カーネル再起動してください。

In [ ]:
!pip install wandb -qqq

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

MNISTは、手書きの数字を分類するために広く使われているデータセットである。このデータセットは、手書き数字の28x28ピクセルのグレースケール画像70,000枚から構成される。このデータセットは60,000枚の学習画像と10,000枚のテスト画像に分割される。10個のクラスがあります（10桁の数字にそれぞれ1つずつ）。このチュートリアルでは、PyTorchを使ってSageMaker上でMNISTモデルを学習・テストする方法を紹介します。


For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [ ]:
#!pip install -U sagemaker==2.87.0

In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist-wandb-studio'

role = sagemaker.get_execution_role()

In [2]:
sagemaker.__version__

'2.77.1'

## Data
### Getting the data



In [3]:
from torchvision.datasets import MNIST
from torchvision import transforms

MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]

MNIST(
    'data',
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [4]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-ap-northeast-1-201228711527/sagemaker/DEMO-pytorch-mnist-wandb-studio


In [5]:
inputs

's3://sagemaker-ap-northeast-1-201228711527/sagemaker/DEMO-pytorch-mnist-wandb-studio'

## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [6]:
!pygmentize src/mnist.py

import argparse
import json
import logging
import os
import sys
import wandb

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = 

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [7]:
import wandb
wandb.login()
settings = wandb.setup().settings
current_api_key = wandb.wandb_lib.apikey.api_key(settings=settings)

/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Curren

In [8]:
from sagemaker.pytorch import PyTorch
'''
estimator = PyTorch(entry_point='mnist.py',
                    source_dir="src",
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type='ml.c5.2xlarge',
                    hyperparameters={
                        'epochs': 1,
                        'backend': 'gloo'
                    },
                   environment={"WANDB_API_KEY": current_api_key})
'''

'\nestimator = PyTorch(entry_point=\'mnist.py\',\n                    source_dir="src",\n                    role=role,\n                    py_version=\'py3\',\n                    framework_version=\'1.8.0\',\n                    instance_count=1,\n                    instance_type=\'ml.c5.2xlarge\',\n                    hyperparameters={\n                        \'epochs\': 1,\n                        \'backend\': \'gloo\'\n                    },\n                   environment={"WANDB_API_KEY": current_api_key})\n'

学習ジョブのエスティメーターを定義


In [17]:
estimator = PyTorch(entry_point='mnist.py',
                    source_dir="src",
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type='ml.c5.2xlarge',
                    hyperparameters={
                        'epochs': 100,
                        'backend': 'gloo'
                    },
                    metric_definitions=[
                        {'Name': 'training/loss', 'Regex': 'Loss: (.*?)#015'},
                        #{'Name': 'Validation Loss', 'Regex': 'val_loss: (.*?);'},
                        {'Name': 'testing/loss', 'Regex': 'Average loss: (.*?),'},
                        {'Name': 'Epoch', 'Regex': 'Epoch: (.*?) '},
                        {'Name': 'epoch', 'Regex': 'Epoch: (.*?) '},
                    ],
                    environment={"WANDB_API_KEY": current_api_key})

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


この設定はStepFunctionsでパイプラインを作るとき、MLOpsエンジニア側の作業となります。

In [19]:
from smexperiments.experiment import Experiment
import boto3
sm = boto3.Session().client('sagemaker')
from smexperiments.tracker import Tracker
import time
from smexperiments.trial import Trial

# experimentの作成
experiment = Experiment.create(experiment_name="mlops-experiment1002", 
                               description="Sample Experiments for MLOps.", 
                               sagemaker_boto_client=sm)

# trackerの作成
with Tracker.create(display_name=f"tracker-{int(time.time())}", sagemaker_boto_client=sm) as tracker:
    tracker.log_input(name="input-dataset-dir", media_type="s3/uri", value='s3://mlops/input/')
    tracker.log_input(name="output-dataset-dir", media_type="s3/uri", value='s3://mlops/output/')

# trialの作成
trial_name = f"training-job-{int(time.time())}"
experiments_trial = Trial.create(
    trial_name=trial_name,
    experiment_name=experiment.experiment_name,
    sagemaker_boto_client=sm,
)

# trial_componentの付与
#sample_trial_component = tracker.trial_component
#experiments_trial.add_trial_component(sample_trial_component)

In [20]:
# trial_componentの付与
estimator_trial_component = tracker.trial_component
experiments_trial.add_trial_component(estimator_trial_component)

In [21]:
estimator.fit({'training': inputs},
    # trialの情報を指定
    experiment_config={
        "ExperimentName": experiment.experiment_name,
        "TrialName": experiments_trial.trial_name,
        "TrialComponentDisplayName": estimator_trial_component.display_name,
    },
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2022-05-31-05-11-19-197


2022-05-31 05:11:20 Starting - Starting the training job...
2022-05-31 05:11:44 Starting - Preparing the instances for trainingProfilerReport-1653973879: InProgress
......
2022-05-31 05:12:49 Downloading - Downloading input data...
2022-05-31 05:13:09 Training - Downloading the training image...
2022-05-31 05:13:50 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-31 05:13:52,501 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-31 05:13:52,503 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-05-31 05:13:52,511 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-31 05:13:52,516 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-31 05:13:52,850 sagemaker-training-toolkit INFO     Install

In [ ]:
#estimator.fit({'training': inputs})

## 複数ジョブ発行

In [16]:
'''
lrs = [0.01, 0.001, 0.05, 0.1, 0.2]
for lr in lrs:
    estimator = PyTorch(entry_point='mnist.py',
                        source_dir="src",
                        role=role,
                        py_version='py3',
                        framework_version='1.8.0',
                        instance_count=1,
                        instance_type='ml.c5.2xlarge',
                        hyperparameters={
                            'epochs': 10,
                            'backend': 'gloo',
                            'lr': lr,
                        },
                       environment={"WANDB_API_KEY": current_api_key})
    estimator.fit({'training': inputs}, wait=False)
'''

'\nlrs = [0.01, 0.001, 0.05, 0.1, 0.2]\nfor lr in lrs:\n    estimator = PyTorch(entry_point=\'mnist.py\',\n                        source_dir="src",\n                        role=role,\n                        py_version=\'py3\',\n                        framework_version=\'1.8.0\',\n                        instance_count=1,\n                        instance_type=\'ml.c5.2xlarge\',\n                        hyperparameters={\n                            \'epochs\': 10,\n                            \'backend\': \'gloo\',\n                            \'lr\': lr,\n                        },\n                       environment={"WANDB_API_KEY": current_api_key})\n    estimator.fit({\'training\': inputs}, wait=False)\n'

In [15]:
lrs = [0.01, 0.001, 0.05, 0.1, 0.2]
for lr in lrs:
    estimator = PyTorch(entry_point='mnist.py',
                        source_dir="src",
                        role=role,
                        py_version='py3',
                        framework_version='1.8.0',
                        instance_count=1,
                        instance_type='ml.c5.2xlarge',
                        hyperparameters={
                            'epochs': 100,
                            'backend': 'gloo',
                            'lr': lr,
                        },
                        metric_definitions=[
                            {'Name': 'training/loss', 'Regex': 'Loss: (.*?)#015'},
                            #{'Name': 'Validation Loss', 'Regex': 'val_loss: (.*?);'},
                            {'Name': 'test/loss', 'Regex': 'Average loss: (.*?),'},
                        ],
                        environment={"WANDB_API_KEY": current_api_key})
    
    estimator.fit({'training': inputs},
        # trialの情報を指定
        experiment_config={
            "ExperimentName": experiment.experiment_name,
            "TrialName": experiments_trial.trial_name,
            "TrialComponentDisplayName": estimator_trial_component.display_name,
        },
        wait=False
    )

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2022-05-31-04-08-46-144
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2022-05-31-04-08-47-368
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2022-05-31-04-08-50-231
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2022-05-31-0